In [ ]:
####Sukhdeep Kaur (March 4th, 2025)
##Project 2

import sys ##Leanred from the internet.
import time
import asyncio
import threading
import multiprocessing


MAX_SAFE_N= 100_000 ###Learned about this from the internet. 
##To stop my computer from freezing, but it still doesn't work.

def is_prime(n):
    if n<=1:
        return False
    for i in range(2, int(n**0.5)+1):
        if n % i ==0:
            return False
    return True

def find_highest_prime():
    start_time=time.time()
    number =0
    highest=0
    print("Finding the highest prime for 3 minutes.")

    while time.time() - start_time < 180:
        if is_prime(number):
            highest= number
        number +=1

    print("Highest prime is:", highest)
    return highest

def fibonacci(n):
    if n> MAX_SAFE_N:
        print(f"Fibonacci({n}) skipped (too large to calculate)")
        return None
    a,b =0, 1
    for _ in range(n):
        a,b =b, a+b
    print(f"Fibonacci({n}) calculated")
    return a

def factorial(n):
    if n> MAX_SAFE_N:
        print(f"Factorial({n})skipped (too large to calculate)")
        return None
    result=1
    for i in range(2, n+1):
        result *= i
    print(f"Factorial({n}) calculated")
    return result

def run_threads():
    print("\n Threaded calculating")
    prime= find_highest_prime()

    fib_thread=threading.Thread(target=fibonacci, args=(prime,))
    fac_thread= threading.Thread(target=factorial, args=(prime,))

    fib_thread.start()
    fac_thread.start()

    fib_thread.join()
    fac_thread.join()

    print("Threading result:")

def prime_process(queque):
        result= find_highest_prime()
        queque.put(result)


def run_multi():
    print("\n Multiprocessing calculating")
    
    queue= multiprocessing.Queue()
    process= multiprocessing.Process(target= prime_process, args=(queue,))
    process.start()
    process.join()

    prime= queue.get()

    fib_thread= threading.Thread(target=fibonacci, args=(prime,))
    fac_thread= threading.Thread(target= factorial, args=(prime,))

    fib_thread.start()
    fac_thread.start()

    fib_thread.join()
    fac_thread.join()
    
    print("Multiprocessing Result:")

async def run_async():
    print("\n Async calculating")

    loop= asyncio.get_event_loop()
    prime = await loop.run_in_executor(None, find_highest_prime)

    fib_task=loop.run_in_executor(None, fibonacci, prime)
    fac_task= loop.run_in_executor(None, factorial, prime)

    await asyncio.gather(fib_task, fac_task)

    print("Async result:")

if __name__=="__main__":
    sys.setrecursionlimit(10000) ##Learned from the internet.

    run_threads()
    run_multi()
    asyncio.run(run_async())

###Gets stuck at multiprocessiung and skips fibonacci and factorial becaause of MAX_SAVE_N



In [ ]:
##Sukhdeep Kaur (March 4th, 2025)
##Project 2
## Second attempt 
import time
import math
import asyncio
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

def check_range(start, end):
    highest_prime = 0
    for num in range(start, end):
        if num > 2 and num % 2 == 0:
            continue
        if is_prime(num):
            highest_prime = num
    return highest_prime

def find_highest_prime_multiprocessing():
    with Pool() as pool:
        start_time = time.time()
        highest_prime = 0
        num = 2
        chunk_size = 10000

        while time.time() - start_time < 180:
            ranges = [(start, start + chunk_size) for start in range(num, num + chunk_size * pool._processes, chunk_size)]
            results = pool.map(check_range, ranges)
            highest_prime = max(highest_prime, *results)
            num += chunk_size * pool._processes
    return highest_prime

def fibonacci(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

def factorial(n):
    return math.factorial(n)

async def async_calculations(highest_prime, number_for_factorial):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        fibonacci_task = loop.run_in_executor(executor, fibonacci, highest_prime)
        factorial_task = loop.run_in_executor(executor, factorial, number_for_factorial)
        fib_result, fact_result = await asyncio.gather(fibonacci_task, factorial_task)
    return fib_result, fact_result

def main():
    print("Finding the highest prime number in 3 minutes using multiprocessing...")

    highest_prime = find_highest_prime_multiprocessing()
    print(f"Highest Prime: {highest_prime}")

    number_for_factorial = 2
    print("Calculating Fibonacci and Factorial together using async and threading...")
    
    loop = asyncio.get_event_loop()
    fib_result, fact_result = loop.run_until_complete(async_calculations(highest_prime, number_for_factorial))
    print(f"Fibonacci of {highest_prime}: {fib_result}")
    print(f"Factorial of {number_for_factorial}: {fact_result}")

if __name__ == "__main__":
    main()